In [1]:
%pwd

'/home/swyoo/DBHW'

In [2]:
%%writefile main.py

#######################################################################
# # Connect to mysql
# # Output: con obj, cursor obj
#######################################################################
IP = "s.snu.ac.kr"
ID = "ADB2018_26190"
PW = "ADB2018_26190"
DBase = "ADB2018_26190"
import pymysql.cursors
import pandas as pd
import nltk
import math 

con = pymysql.connect(host = IP, 
                            user = ID,
                            password = PW,
                            db = DBase,
                            charset = 'utf8mb4')
# con = pymysql.connect(host = 'localhost', 
#                             user = 'root',
#                             password = 'root',
#                             db = dbname,
#                             charset = 'utf8mb4')
cursor = con.cursor()

print("building tables...")
# # Before executing this file, create wiki, link table in DBase using mysql-workbench  
#######################################################################
# # Make inverted index table 
# # Input: wiki table 
# # Output: InvertedIndex table(term, id, title, freq)
#######################################################################
sql = "select * from wiki"
cursor.execute(sql)
result_wiki = cursor.fetchall()

sql = "select count(*) from information_schema.tables where (table_schema = %s) and (table_name = %s)"
cursor.execute(sql, (DBase, "InvertedIndex"))

if cursor.fetchall()[0][0] == 0: 
    # Create inverted index table 
    sql = "create table InvertedIndex (term varchar(255) not null, id int(11) not null, freq int(11) not null)"
    cursor.execute(sql)
    # # Clean inverted index table
    # sql = "delete from InvertedIndex" 
    # cursor.execute(sql)

    # insertion from wiki table 
    sql = "insert into InvertedIndex (term,id,freq) values (%s,%s,%s)"
    for Doc in result_wiki:
    #     print(type(Doc)) #tuple (id, doc_name, doc_script)
        tokens = nltk.word_tokenize(Doc[2].lower())
        fdist = nltk.FreqDist(tokens) # dictionary {term:freq, ... }
        for term, freq in fdist.items():
            cursor.execute(sql,(term, Doc[0], freq))
    #     print(fdist)
    con.commit()
    
#cursor.execute("drop table InvertedIndex")

print("ready to search...")
#######################################################################
# # PageRank Score Calculation
# # Input: link table 
# # Output: PageRank list R(id, PageRank score)
#######################################################################
# print("calculating pagerank score ...")
import numpy as np
# Get Ni for each
sql = "select id_from, count(*) as outgoing from link group by id_from order by id_from"
cursor.execute(sql)
FromNiInfo = cursor.fetchall()

Ni_dict = {}
for idx, FromNi in enumerate(FromNiInfo):
#     print(FromNi[0], FromNi[1])
    id_from = FromNi[0]
    Ni = FromNi[1]
    Ni_dict[id_from] = Ni

# Get N and id_all(sorted order) 
sql = "select distinct id_from from link order by id_from"
cursor.execute(sql)
SetOfFromInfo = cursor.fetchall()

sql = "select distinct id_to from link order by id_to"
cursor.execute(sql)
SetOfToInfo = cursor.fetchall()

SetOfFrom = set()
SetOfTo = set()

for idx, From in enumerate(SetOfFromInfo):
#     print(type(From[0]))
    SetOfFrom.add(From[0])
    
for idx, To in enumerate(SetOfToInfo):
#     print(type(From[0]))
    SetOfTo.add(To[0])
    
id_all = sorted(SetOfFrom.union(SetOfTo))
N = len(id_all)

# Get N_idx and N_idx_inverse; it means dictionary[Doc.id] = index of Transition Matrix or State Matrix    
N_idx = {}
N_idx_inverse = {}
for idInfo, idx in enumerate(id_all):
#     print(idx, idInfo)
    N_idx[idx] = idInfo
    N_idx_inverse[idInfo] = idx

# Get SateMatrix S; check whether existing from j to i link  
S = np.zeros((N,N)) # from j to i info : S[i][j]
sql = "select * from link order by id_from"
cursor.execute(sql)
FromToInfo = np.array(cursor.fetchall())
for fromto in FromToInfo:
    id_to = fromto[1]
    id_from = fromto[0]
    S[N_idx[id_to]][N_idx[id_from]] = 1

# Get Transition Matrix M and Score Vector R 
M = np.zeros((N,N)) # from j to i info : M[i][j]
for _, id_from in enumerate(sorted(SetOfFrom)):
    for _, id_to in enumerate(id_all):
        # if link id_from to id_to exists
        if S[N_idx[id_to]][N_idx[id_from]] != 0:
            M[N_idx[id_to]][N_idx[id_from]] = 1/Ni_dict[id_from]

# PageRank Algorithm
# Input: Station Matrix S, Transition Matrix T, RankVector R 
# Output: updated RankVector R 
delta = 0.15
elipslion = 1e-8
# R = np.ones((N,1))*(1/N)
R = np.ones((N,1))
K = np.ones((N,1))*(delta/N)
# R = delta * np.matmul(M,prevR) + K
iteration = 0
distance = 100
while distance > elipslion:
#     print("iteration", iteration, "...")
    prevR = R
    R = delta * np.matmul(M,R) + K
    iteration = iteration + 1
    distance = np.linalg.norm(R-prevR)
#     print("distance = ",np.linalg.norm(R-prevR))

#######################################################################
# # TF-IDF Score calculation
# # Input: InvertedIndex table, wiki table, query(string)
# # Output: TF-IDF list TFIDF(id, title, TF-IDF score) 
#######################################################################
def TFIDFscore(Nd, Ndt, Nt):
        return math.log1p(Ndt/Nd)*(1/Nt)
while True:
    
    # Get input from console
    print("2018-26190>",end = '')
    query = input()
    if query == "exit()":
        break
    querys = query.lower().split()
    
    TFIDFs_idx = {}
    TFIDFs_idx_inverse = {}
    TFIDFs = []
    iteration = 0
    for query in querys:
        #######################################################################
        # # TF-IDF Score calculation for each word 
        #######################################################################
        sql = "select sum(freq),id from InvertedIndex where id in (select id from InvertedIndex where term = %s) group by id order by id"
        cursor.execute(sql,query)
        NdInfo = cursor.fetchall()

        sql = "select freq, id, term from InvertedIndex where term = %s order by id"
        cursor.execute(sql,query)
        NdtInfo = cursor.fetchall()

        sql = "select count(*) from InvertedIndex where term = %s"
        cursor.execute(sql,query)
        Nt = cursor.fetchall()[0][0]

        TFIDF = []
        for i in range(len(NdtInfo)):
            if NdInfo[i][1] == NdtInfo[i][1]:
                Nd = NdInfo[i][0]
                Ndt = NdtInfo[i][0]
                id = NdInfo[i][1]
        #         print(id," ",math.log1p(Ndt/Nd)*(1/Nt))
        #         TFIDF.append((id, math.log1p(Ndt/Nd)*(1/Nt)))
                TFIDF.append((id, TFIDFscore(Nd, Ndt, Nt)))
        TFIDFs.append(TFIDF)
        TFIDFs_idx[query] = iteration
        TFIDFs_idx_inverse[iteration] = query
        iteration = iteration + 1

    # Get title dictionary     
    sql = "select id,title from wiki order by id"
    cursor.execute(sql)
    id_title = cursor.fetchall()
    id_title_dictionary = {}
    for _, idtitle in enumerate(id_title):
        v_id = idtitle[0]
        v_title = idtitle[1]
        id_title_dictionary[v_id] = v_title

    # Union TFIDFs
    TFIDFsetlist = []      
    for query in querys:
        temp = np.array(TFIDFs[TFIDFs_idx[query]])
        if len(temp) == 0:
            idsForOneQuery = temp.astype(int)
        else:
            idsForOneQuery = temp[...,0].astype(int)
        TFIDFsetlist.append(set(idsForOneQuery))

    UnionId = []
    UnionId_idx = {}
    UnionId_idx_inverse = {}
    for TFIDFset in TFIDFsetlist:
        UnionId = list(set(UnionId)|set(TFIDFset))

    UnionId.sort()
    iteration = 0;
    for idvalue in UnionId:
        UnionId_idx[idvalue] = iteration
        UnionId_idx_inverse[iteration] = idvalue
        iteration = iteration + 1

    # Get TFIDF Score Matrix SM
    SM = np.zeros((len(querys), len(UnionId)))
    for row in range(len(querys)):
        LenOfcols = len(TFIDFs[row])
    #     print(LenOfcols)
        for j in range(LenOfcols): # j means each query's index of id
    #         print(TFIDFs[row][j][0], TFIDFs[0][j][1], TFIDFs[0][j][2])  # 
            col = UnionId_idx[TFIDFs[row][j][0]]
    #         title = TFIDFs[row][j][1]
            score = TFIDFs[row][j][1]
            SM[row][col] = score
    # SM
    #######################################################################
    # # Get Top K list
    # # Input: TFIDF Score Matrix SM, PageRank list R
    # # Output: QAList(sorted order by TFIDF score * PageRank score)
    #######################################################################
    QAList = []
    for it in UnionId_idx:
        Id = it
        UnionTFIDFScore = SM[...,UnionId_idx[it]].sum()
        Uniontitle = id_title_dictionary[it] 
        PrankScore = R[N_idx[Id]][0]
        QAList.append((Id, Uniontitle, UnionTFIDFScore, PrankScore))

    def SortCriteria(item):
        return item[2]*item[3]
    QAList.sort(key= SortCriteria, reverse= True)

    strFormat = '%-10s%-60s%-20s%-20s\n'
    strOut = strFormat % ('id', 'title', 'TF-IDF', 'PageRank')
    iteration = 0;
    for ans in QAList:
        if iteration < 10:
#             print(ans[0], ans[1], format(ans[2],"10.2e"), format(ans[3],"10.2e"))
            strOut += strFormat %(ans[0], ans[1], format(ans[2],"10.2e"), format(ans[3],"10.2e"))
        else:
           break
        iteration = iteration + 1
    print(strOut)    

con.close()

Overwriting main.py


In [3]:
%run main.py

building tables...
ready to search...
2018-26190>the president
id        title                                                       TF-IDF              PageRank            
44392605  President_of_the_Quorum_of_the_Twelve                         1.04e-03            2.34e-05          
8799424   Bishop_(Catholic_Church)                                      1.47e-04            9.55e-05          
34999343  Barry_Mendelson                                               4.19e-04            2.34e-05          
19605700  East_Asia                                                     1.28e-04            7.52e-05          
47510823  Department_of_State_Affairs                                   2.61e-04            2.95e-05          
28473028  Anti-American_sentiment_in_Pakistan                           3.21e-04            2.34e-05          
48652315  Presidency_of_Boris_Yeltsin                                   3.10e-04            2.34e-05          
47802357  1998_New_Zealand_NBL_season            

# Connect to mysql 

In [5]:
# connection = pymysql.connect(host = 'localhost', 
#                             user = 'root',
#                             password = 'root',
#                             db = 'swyoo',
#                             charset = 'utf8mb4')
# try:
#     cursor = connection.cursor()
#     print("# tables:", cursor.execute("show tables"))
#     sql = "select * from wiki"
#     print("# row of wiki:", cursor.execute(sql))
#     result_wiki = cursor.fetchall()
#     sql = "select * from link"
#     print("# row of link:", cursor.execute(sql))
#     result_link = cursor.fetchall()
#     print("[Sample of dataset] ")
#     print(" from wiki .... ")
#     print(" id: ", result_wiki[0][0])
#     print(" title:", result_wiki[0][1])
#     print(" from link .... ")
#     print(" (from , to):", result_link[0])
# #     for row_data in result:
# #         print(row_data)        
# finally:
#     connection.close()

In [6]:
# import nltk
# nltk.download('punkt')

In [7]:
import pymysql.cursors
import pandas as pd
import nltk
con = pymysql.connect(host = 'localhost', 
                            user = 'root',
                            password = 'root',
                            db = 'swyoo',
                            charset = 'utf8mb4')

In [8]:
cursor = con.cursor()
pd.read_sql("show tables",con)

,Tables_in_swyoo
0,InvertedIndex
1,link
2,wiki


# Parsing wiki table and Make InvertedIndex table

In [9]:
sql = "select * from wiki"
cursor.execute(sql)
result_wiki = cursor.fetchall()

In [10]:
# test tokenization 
# tokens = nltk.word_tokenize(result_wiki[0][2])
# fdist = nltk.FreqDist(tokens)

In [11]:
# # Make inverted index table 
# sql = "create table InvertedIndex (term varchar(255) not null, id int(11) not null, freq int(11) not null)"
# cursor.execute(sql)

# Clean inverted Index table 
# cursor.execute("delete from InvertedIndex")
# con.commit()

In [12]:
# sql = "select * from InvertedIndex"
# pd.read_sql(sql,con)

In [13]:
# sql = "insert into InvertedIndex (term,id,freq) values (%s,%s,%s)"
# for Doc in result_wiki:
# #     print(type(Doc)) #tuple (id, doc_name, doc_script)
#     tokens = nltk.word_tokenize(Doc[2].lower())
#     fdist = nltk.FreqDist(tokens) # dictionary {term:freq, ... }
#     for term, freq in fdist.items():
#         cursor.execute(sql,(term, Doc[0], freq))
# #     print(fdist)
# con.commit()

In [14]:
# sql = "select * from InvertedIndex where term = 'man' order by id"
# pd.read_sql(sql,con)

# TF-IDF score calculation 

In [25]:
# assume that given query word "query"
query = input()

president


In [26]:
# sql = "select freq, id, term from InvertedIndex where term = '"+ query +"'order by id"
# pd.read_sql(sql,con)

In [27]:
sql = "select sum(freq),id from InvertedIndex where id in (select id from InvertedIndex where term = %s) group by id order by id"
cursor.execute(sql,query)
NdInfo = cursor.fetchall()
# # NdInfo

In [28]:
# sql = "select freq, id, term from InvertedIndex where term = '"+ query +"'order by id"
# pd.read_sql(sql,con)

In [29]:
sql = "select freq, id, term from InvertedIndex where term = %s order by id"
cursor.execute(sql,query)
NdtInfo = cursor.fetchall()
# #NdtInfo # tuple (freq, id)

In [30]:
# sql = "select count(distinct id) from InvertedIndex where term = '"+ query +"'"
# pd.read_sql(sql,con)

In [31]:
sql = "select count(*) from InvertedIndex where term = %s"
cursor.execute(sql,query)
Nt = cursor.fetchall()[0][0]
# Nt

In [32]:
print(len(NdInfo))
print(len(NdtInfo))
print(Nt)

46
46
46


In [33]:
# assume that given query word "query"
sql = "select sum(freq),id from InvertedIndex where id in (select id from InvertedIndex where term = %s) group by id order by id"
cursor.execute(sql,query)
NdInfo = cursor.fetchall()

sql = "select freq, id, term from InvertedIndex where term = %s order by id"
cursor.execute(sql,query)
NdtInfo = cursor.fetchall()

sql = "select count(*) from InvertedIndex where term = %s"
cursor.execute(sql,query)
Nt = cursor.fetchall()[0][0]

sql = "select title,id from wiki where id in (select id from InvertedIndex where term = %s) order by id"
cursor.execute(sql,query)
titles = cursor.fetchall()

import math 

def TFIDFscore(Nd, Ndt, Nt):
    return math.log1p(Ndt/Nd)*(1/Nt)

TFIDF = []
for i in range(len(NdtInfo)):
    if NdInfo[i][1] == NdtInfo[i][1] & NdInfo[i][1] == titles[i][1]:
        Nd = NdInfo[i][0]
        Ndt = NdtInfo[i][0]
        id = NdInfo[i][1]
        title = titles[i][0]
#         print(id," ",math.log1p(Ndt/Nd)*(1/Nt))
#         TFIDF.append((id, math.log1p(Ndt/Nd)*(1/Nt)))
        TFIDF.append((id, title, TFIDFscore(Nd, Ndt, Nt)))

# def takeSecond(item):
#     return item[2]

# TFIDF.sort(key=takeSecond, reverse = True)

In [34]:
print(len(TFIDF))
print(len(titles))

46
46


In [35]:
TFIDF

[(416890, 'Chris_Coleman_(footballer)', 2.6462730516341183e-05),
 (1259501, '1874_in_rail_transport', 6.557812042229655e-05),
 (1585224, 'Alex_Smith', 4.359597013464207e-06),
 (2319776, 'Lunar_effect', 9.49514338518773e-06),
 (3489138, 'Janice_Hahn', 1.3523565247806368e-05),
 (3574725, 'Bianca_Montgomery', 2.401981156499539e-06),
 (3979359, 'Hans_J%C3%B8rgen_Darre-Jenssen', 7.708919579892524e-05),
 (4452121, 'Randy_Hahn', 6.460370411211935e-05),
 (6301341, 'Colina%2C_Chile', 8.927786853594372e-05),
 (6749015, 'Tommy_Koh', 5.038039435570704e-05),
 (10084456, 'Boston_University_School_of_Medicine', 2.96375375622628e-05),
 (11142981, 'John_Michael_Miller', 5.361070641725004e-05),
 (12706409, 'David_Martin_(artist)', 7.092707901774742e-05),
 (13534959, 'USS_Trumbull_(1800)', 4.581483346879813e-05),
 (13601579, 'Quentin_L._Cook', 0.0001478858113641852),
 (14472168, 'Estanislao_Esteban_Karlic', 5.931553593921787e-05),
 (15092026, 'Yohanan_Plesner', 3.2567991761348e-05),
 (15923521, 'Darney',

In [36]:
# iteration = 0;
# for e in TFIDF:
#     if iteration < len(TFIDF)/5:
#         print(e)
#     else:
#        break
#     iteration = iteration + 1

# PageRank score calculation

In [37]:
import numpy as np

In [38]:
# Get Ni for each
sql = "select id_from, count(*) as outgoing from link group by id_from order by id_from"
cursor.execute(sql)
FromNiInfo = cursor.fetchall()

Ni_dict = {}
for idx, FromNi in enumerate(FromNiInfo):
#     print(FromNi[0], FromNi[1])
    id_from = FromNi[0]
    Ni = FromNi[1]
    Ni_dict[id_from] = Ni
# Ni_dict  

In [39]:
# Get N and id_all(sorted order) 
sql = "select distinct id_from from link order by id_from"
cursor.execute(sql)
SetOfFromInfo = cursor.fetchall()

sql = "select distinct id_to from link order by id_to"
cursor.execute(sql)
SetOfToInfo = cursor.fetchall()

SetOfFrom = set()
SetOfTo = set()

for idx, From in enumerate(SetOfFromInfo):
#     print(type(From[0]))
    SetOfFrom.add(From[0])
    
for idx, To in enumerate(SetOfToInfo):
#     print(type(From[0]))
    SetOfTo.add(To[0])

print(len(SetOfFrom))
print(len(SetOfTo))
# print(len(SetOfFrom.intersection(SetOfTo)))
# print("============")
# print(len(SetOfFrom) + len(SetOfTo) - len(SetOfFrom.intersection(SetOfTo)))
id_all = sorted(SetOfFrom.union(SetOfTo))
N = len(id_all)
print(N)

4398
3625
6403


In [40]:
# Get N_idx and N_idx_inverse; it means dictionary[Doc.id] = index of Transition Matrix or State Matrix    
N_idx = {}
N_idx_inverse = {}
for idInfo, idx in enumerate(id_all):
#     print(idx, idInfo)
    N_idx[idx] = idInfo
    N_idx_inverse[idInfo] = idx
print(len(N_idx))
print(len(N_idx_inverse))
# N_idx

6403
6403


In [41]:
# Get SateMatrix S; check whether existing from j to i link  
S = np.zeros((N,N)) # from j to i info : S[i][j]
# id_from = 17143
# id_to = 2727051
sql = "select * from link order by id_from"
cursor.execute(sql)
FromToInfo = np.array(cursor.fetchall())
for fromto in FromToInfo:
    id_to = fromto[1]
    id_from = fromto[0]
    S[N_idx[id_to]][N_idx[id_from]] = 1
# S[N_idx[2727051]][N_idx[17143]]

In [42]:
# Get Transition Matrix M and Score Vector R 
M = np.zeros((N,N)) # from j to i info : M[i][j]
# print(M.shape)
for _, id_from in enumerate(sorted(SetOfFrom)):
    for _, id_to in enumerate(id_all):
        # if link id_from to id_to exists
        if S[N_idx[id_to]][N_idx[id_from]] != 0:
            M[N_idx[id_to]][N_idx[id_from]] = 1/Ni_dict[id_from]

In [43]:
## Error was found; N_idx have to be fixed(that is, link data should be fixed)
# I found link dataset's error! duplicates exist ===>> Dataset is fixed!! 
for it in N_idx:
    print(M[...,N_idx[it]].sum())

0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
0.0
1

0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0


0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0


In [44]:
# PageRank Algorithm
delta = 0.15
elipslion = 1e-8
# R = np.ones((N,1))*(1/N)
R = np.ones((N,1))
K = np.ones((N,1))*(delta/N)
# R = delta * np.matmul(M,prevR) + K
iteration = 0
distance = 100
while distance > elipslion:
    print("iteration", iteration, "...")
    prevR = R
    R = delta * np.matmul(M,R) + K
    iteration = iteration + 1
    distance = np.linalg.norm(R-prevR)
    print("distance = ",np.linalg.norm(R-prevR))

# Prank = []
# for idx, r in enumerate(R):
#     Prank.append((N_idx_inverse[idx], r[0]))

# def takeSecond(item):
#     return item[1]

# Prank.sort(key= takeSecond, reverse= True)
# Prank
R

iteration 0 ...
distance =  74.10893977199761
iteration 1 ...
distance =  19.654939936357895
iteration 2 ...
distance =  1.317147357409797
iteration 3 ...
distance =  0.16673183038197417
iteration 4 ...
distance =  0.023230458923275472
iteration 5 ...
distance =  0.00363864354638029
iteration 6 ...
distance =  0.0005172302450978651
iteration 7 ...
distance =  8.159564757324994e-05
iteration 8 ...
distance =  1.1621562302864015e-05
iteration 9 ...
distance =  1.8351128469182678e-06
iteration 10 ...
distance =  2.615057805869916e-07
iteration 11 ...
distance =  4.129248301639776e-08
iteration 12 ...
distance =  5.885459257455751e-09


array([[2.69404966e-05],
       [4.82879119e-05],
       [3.38125195e-05],
       ...,
       [2.69404966e-05],
       [2.53259663e-05],
       [2.34265188e-05]])

In [45]:
# Prank = []
# for idx, r in enumerate(R):
#     Prank.append((N_idx_inverse[idx], r[0]))
# `
# def takeSecond(item):
#     return item[1]

# Prank.sort(key= takeSecond, reverse= True)
# # Prank

In [46]:
print(len(R))
print(len(TFIDF))

6403
46


In [47]:
QAList = [] # DocScore = TFIDFScore * PrankScore
for row in TFIDF:
#      print(row)
    PrankScore = R[N_idx[row[0]]][0]
#     print(row[0], row[1], PrankScore, row[2])
    QAList.append((row[0], row[1], PrankScore, row[2]))
    
def SortCriteria(item):
    return item[2]*item[3]
QAList.sort(key= SortCriteria, reverse= True)

strFormat = '%-10s%-50s%-20s%-10s\n'
strOut = strFormat % ('id', 'title', 'TF-IDF', 'PageRank')
iteration = 0;
for ans in QAList:
    if iteration < 10:
#         print(ans[0], ans[1], ans[2], ans[3])
        strOut += strFormat %(ans[0], ans[1], format(ans[2],"10.2e"), format(ans[3],"10.2e"))
    else:
       break
    iteration = iteration + 1

print(strOut)


id        title                                             TF-IDF              PageRank  
44392605  President_of_the_Quorum_of_the_Twelve               2.34e-05            7.28e-04
34999343  Barry_Mendelson                                     2.34e-05            3.24e-04
28473028  Anti-American_sentiment_in_Pakistan                 2.34e-05            1.85e-04
22553537  Yousef_Pashtun                                      2.61e-05            1.44e-04
13601579  Quentin_L._Cook                                     2.34e-05            1.48e-04
31002740  Westye_Parr_Egeberg                                 2.53e-05            1.36e-04
28246228  St._Louis_Jewish_Light                              2.69e-05            1.02e-04
37619696  Falconar_Avia                                       3.12e-05            8.75e-05
6301341   Colina%2C_Chile                                     2.76e-05            8.93e-05
48652315  Presidency_of_Boris_Yeltsin                         2.34e-05            1.04e-04

# Multiple words case

In [48]:
query = input()

I love you


In [49]:
querys = query.lower().split()
querys

['i', 'love', 'you']

In [50]:
# for i in range(len(querys)):
#     print(i)
for query in querys:
    print(query)

i
love
you


In [51]:
def TFIDFscore(Nd, Ndt, Nt):
        return math.log1p(Ndt/Nd)*(1/Nt)

TFIDFs_idx = {}
TFIDFs_idx_inverse = {}
TFIDFs = []
iteration = 0
for query in querys:
    #######################################################################
    # # TF-IDF Score calculation for one word 
    #######################################################################
    sql = "select sum(freq),id from InvertedIndex where id in (select id from InvertedIndex where term = %s) group by id order by id"
    cursor.execute(sql,query)
    NdInfo = cursor.fetchall()

    sql = "select freq, id, term from InvertedIndex where term = %s order by id"
    cursor.execute(sql,query)
    NdtInfo = cursor.fetchall()

    sql = "select count(*) from InvertedIndex where term = %s"
    cursor.execute(sql,query)
    Nt = cursor.fetchall()[0][0]
    
#     print(len(NdInfo))
#     print(len(NdtInfo))
#     print(Nt)
    TFIDF = []
    for i in range(len(NdtInfo)):
        if NdInfo[i][1] == NdtInfo[i][1]:
            Nd = NdInfo[i][0]
            Ndt = NdtInfo[i][0]
            id = NdInfo[i][1]
    #         print(id," ",math.log1p(Ndt/Nd)*(1/Nt))
    #         TFIDF.append((id, math.log1p(Ndt/Nd)*(1/Nt)))
            TFIDF.append((id, TFIDFscore(Nd, Ndt, Nt)))
    TFIDFs.append(TFIDF)
    TFIDFs_idx[query] = iteration
    TFIDFs_idx_inverse[iteration] = query
    iteration = iteration + 1   

In [52]:
sql = "select id,title from wiki order by id"
cursor.execute(sql)
id_title = cursor.fetchall()
id_title_dictionary = {}
for _, idtitle in enumerate(id_title):
    v_id = idtitle[0]
    v_title = idtitle[1]
    id_title_dictionary[v_id] = v_title
# id_title_dictionary[106791]

In [53]:
# TFIDFs[TFIDFs_idx["wanna"]]
# test = np.array([])
# len(test)

In [54]:
# Union
TFIDFsetlist = []
for query in querys:
    temp = np.array(TFIDFs[TFIDFs_idx[query]])
    if len(temp) == 0:
        idsForOneQuery = temp.astype(int)
    else:
        idsForOneQuery = temp[...,0].astype(int)
    TFIDFsetlist.append(set(idsForOneQuery))

UnionId = []
UnionId_idx = {}
UnionId_idx_inverse = {}
for TFIDFset in TFIDFsetlist:
    print(len(UnionId)) 
    UnionId = list(set(UnionId)|set(TFIDFset))

UnionId.sort()

print(len(UnionId))
iteration = 0;
for idvalue in UnionId:
    UnionId_idx[idvalue] = iteration
    UnionId_idx_inverse[iteration] = idvalue
    iteration = iteration + 1
    
# Get TFIDF Score Matrix SM
SM = np.zeros((len(querys), len(UnionId)))
print(SM.shape)
for row in range(len(querys)):
    LenOfcols = len(TFIDFs[row])
#     print(LenOfcols)
    for j in range(LenOfcols): # j means each query's index of id
#         print(TFIDFs[row][j][0], TFIDFs[0][j][1], TFIDFs[0][j][2])  # 
        col = UnionId_idx[TFIDFs[row][j][0]]
#         title = TFIDFs[row][j][1]
        score = TFIDFs[row][j][1]
        SM[row][col] = score
# SM

0
78
92
106
(3, 106)


In [55]:
# print(len(UnionId_idx))
QAList = []
for it in UnionId_idx:
    Id = it
    UnionTFIDFScore = SM[...,UnionId_idx[it]].sum()
    Uniontitle = id_title_dictionary[it] 
    PrankScore = R[N_idx[Id]][0]
    QAList.append((Id, Uniontitle, UnionTFIDFScore, PrankScore))
#     print(it)
#     print(SM[...,UnionId_idx[it]].sum())

def SortCriteria(item):
    return item[2]*item[3]
QAList.sort(key= SortCriteria, reverse= True)


strFormat = '%-10s%-50s%-20s%-20s\n'
strOut = strFormat % ('id', 'title', 'TF-IDF', 'PageRank')
iteration = 0;
for ans in QAList:
    if iteration < 10:
#         print(ans[0], ans[1], ans[2], ans[3])
        strOut += strFormat %(ans[0], ans[1], format(ans[2],"10.2e"), format(ans[3],"10.2e"))
    else:
       break
    iteration = iteration + 1

# strFormat = '%-10s%-50s%-20s%-20s%-20s\n'
# strOut = strFormat % ('id', 'title', 'TF-IDF', 'PageRank', 'TFIDF*PageRank' )
# iteration = 0;
# for ans in QAList:
#     if iteration < 10:
# #         print(ans[0], ans[1], ans[2], ans[3])
#         strOut += strFormat %(ans[0], ans[1], format(ans[2],"10.2e"), format(ans[3],"10.2e"), format(ans[2]*ans[3],"10.2e"))
#     else:
#        break
#     iteration = iteration + 1

print(strOut)

id        title                                             TF-IDF              PageRank            
36630463  Kuwaiti_Division_I_Basketball_League                1.28e-03            2.69e-05          
44161177  Wila_Willk%27i_(La_Paz)                             4.20e-04            3.22e-05          
7026696   Se_piangi%2C_se_ridi                                4.78e-04            2.34e-05          
2726686   Headfirst_for_Halos                                 4.39e-04            2.34e-05          
32286524  Roll_On_Down_the_Highway                            4.02e-04            2.34e-05          
5882384   Samrat_Upadhyay                                     4.02e-04            2.34e-05          
6241635   Sing_Sang_Song                                      2.64e-04            2.34e-05          
38821243  Monsieur_Minimal                                    2.20e-04            2.34e-05          
12133016  Live_Earth_concert%2C_Tokyo                         2.20e-04            2.34e-05 

In [56]:
# UnionId

In [58]:
# # Test
# print(len(TFIDFs[0]))
# print(len(UnionId))
# j = 77
# print(TFIDFs[0][j][0], TFIDFs[0][j][1], TFIDFs[0][j][2])
# print(UnionId_idx[TFIDFs[0][j][0]])
# # UnionId
# # len(TFIDFs[0])

In [59]:
querys

['i', 'love', 'you']

In [60]:
con.close()

In [61]:
# cursor.execute("drop table wiki")
# cursor.execute("drop table link")
# cursor.execute("drop table InvertedIndex")

In [62]:
# pd.read_sql("show tables", con)